<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2020ccap/2020_0521ccap_softmax_with_temparature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ccap 勉強会資料
- date: 2020-0521
- author: 浅川伸一
- title: 注意に使用されるソフトマックス関数は，温度を付けることができる

たとえば，故障課題において，脳内に想起された回答候補を選択する確率が，その内部刺激表現の強度のみならず，
この温度にも影響を受けると考えれば，故障障害の記述に洞察を与えることができるだろうと考える。




In [0]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.misc import 
from scipy.special import logsumexp

In [0]:
def softmax_wT(x, T=1., axis=None):
    """
    Softmax function

    The softmax function transforms each element of a collection by
    computing the exponential of each element divided by the sum of the
    exponentials of all the elements.  That is, if `x` is a one-dimensional
    numpy array::

        softmax(x) = np.exp(x)/sum(np.exp(x))

    Parameters
    ----------
    x : array_like
        Input array.
    axis : int or tuple of ints, optional
        Axis to compute values along. Default is None and softmax will be
        computed over the entire array `x`.

    Returns
    -------
    s : ndarray
        An array the same shape as `x`. The result will sum to 1 along the
        specified axis.
    """

    return np.exp(x/T - logsumexp(x/T, axis=axis, keepdims=True))

In [0]:
np.random.seed(3)  # 乱数生成器の種を設定

N = 3  # 生成する乱数の個数
raw_data = np.random.rand(N)  
print(raw_data, softmax_wT(raw_data), softmax_wT(raw_data, T=10))

In [0]:
width = 0.3

plt.figure(figsize=(10,8))
plt.bar(x - width, softmax_wT(raw_data, T=0.1), width, label='softmax T=0.1')
plt.bar(x,         softmax_wT(raw_data, T=1), width, label='softmax T=1')
plt.bar(x + width, softmax_wT(raw_data, T=30), width, label='softmax T=30')
plt.legend()
plt.show()